<a href="https://colab.research.google.com/github/chanidapa14092547/229351-StatisticalLearning/blob/main/Lab02_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve

# For Fashion-MNIST
from tensorflow.keras.datasets import fashion_mnist

# For 20 Newsgroups
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## Part 1: Marketing Campaign Dataset - Manual Data Preprocessing & Logistic Regression

### Load the Marketing Campaign Dataset ([Data Information](https://archive.ics.uci.edu/dataset/222/bank+marketing))

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be (`'yes'`) or not (`'no'`) subscribed.

In [2]:
bank_url = 'https://raw.githubusercontent.com/donlap/ds352-labs/main/bank.csv'

df = pd.read_csv(bank_url, sep=';', na_values=['unknown'])
df = df.drop(["emp.var.rate", "cons.price.idx", "cons.conf.idx",	"euribor3m", "nr.employed"], axis=1)
print("Shape of the dataset:", df.shape)
df.head()

Shape of the dataset: (41188, 16)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,no
1,57,services,married,high.school,NaN,no,no,telephone,may,mon,149,1,999,0,nonexistent,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,no


### Data Exploration

In [3]:
print("--- Missing Values Count ---")
print(df.isnull().sum())

--- Missing Values Count ---
age               0
job             330
marital          80
education      1731
default        8597
housing         990
loan            990
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
y                 0
dtype: int64


In [4]:
print("--- Unique Values for Categorical Columns ---")
for col in df.select_dtypes(include='object').columns:
    print(f"\n'{col}' unique values:")
    print(df[col].value_counts(dropna=False)) # Include NaN counts

--- Unique Values for Categorical Columns ---

'job' unique values:
job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
NaN                330
Name: count, dtype: int64

'marital' unique values:
marital
married     24928
single      11568
divorced     4612
NaN            80
Name: count, dtype: int64

'education' unique values:
education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
NaN                     1731
illiterate                18
Name: count, dtype: int64

'default' unique values:
default
no     32588
NaN     8597
yes        3
Name: count, dtype: int64

'housing' unique values:
housing
yes    21576
no     18622
NaN      990
Name: count, dtype: int64


### Data Preprocessing

In [5]:
# Map target variable 'y' to 0 (no) and 1 (yes)
df['y_new'] = df['y'].map({'yes': 1,'no': 0}) # Write your code here


# Drop 'duration' due to data leakage


# Define features (X) and target (y)
y = df['y_new']
x = df.drop(['y', 'y_new'], axis=1)

print(x)
print(y)


       age          job  marital            education default housing loan    contact month day_of_week  duration  campaign  pdays  previous     poutcome
0       56    housemaid  married             basic.4y      no      no   no  telephone   may         mon       261         1    999         0  nonexistent
1       57     services  married          high.school     NaN      no   no  telephone   may         mon       149         1    999         0  nonexistent
2       37     services  married          high.school      no     yes   no  telephone   may         mon       226         1    999         0  nonexistent
3       40       admin.  married             basic.6y      no      no   no  telephone   may         mon       151         1    999         0  nonexistent
4       56     services  married          high.school      no      no  yes  telephone   may         mon       307         1    999         0  nonexistent
...    ...          ...      ...                  ...     ...     ...  ...  

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

x_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
3819,34,blue-collar,married,basic.9y,no,no,no,telephone,may,fri,344,4,999,0,nonexistent
18793,33,technician,married,university.degree,no,yes,no,cellular,jul,thu,52,7,999,0,nonexistent
2126,34,blue-collar,single,basic.6y,NaN,yes,no,telephone,may,mon,891,1,999,0,nonexistent
39389,56,admin.,single,university.degree,no,no,no,cellular,mar,wed,193,1,5,1,success
39000,38,technician,single,university.degree,no,yes,no,cellular,dec,mon,329,2,999,0,nonexistent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7122,41,housemaid,married,university.degree,no,no,no,telephone,may,thu,665,1,999,0,nonexistent
1356,55,housemaid,married,basic.4y,no,yes,no,telephone,may,thu,14,1,999,0,nonexistent
38476,33,admin.,single,high.school,no,no,no,cellular,oct,mon,251,1,999,0,nonexistent
19651,36,admin.,married,high.school,NaN,yes,no,cellular,aug,thu,165,3,999,0,nonexistent


We will apply `StandardScaler()`, `OrdinalEncoder()`, and `OneHotEncoder()` on a few selected columns.

**1. Numerical Feature: `age` and `campaign` (Standard Scaling)**

In [7]:
num_cols_demo = ['age', 'campaign']

scaler = StandardScaler()

x_train[num_cols_demo] = scaler.fit_transform(x_train[num_cols_demo])
x_test[num_cols_demo] = scaler.transform(x_test[num_cols_demo])

# Fit the scaler ONLY on the training data


#X_train_scaled_demo = # Write your code here
#X_test_scaled_demo = # Write your code here

In [8]:
x_train.describe()

,age,duration,campaign,pdays,previous
count,2.883100e+04,28831.000000,2.883100e+04,28831.000000,28831.000000
mean,-2.819399e-16,258.457702,7.886430e-18,962.318338,0.172488
std,1.000017e+00,260.709354,1.000017e+00,187.301605,0.493777
min,-2.214257e+00,0.000000,-5.660315e-01,0.000000,0.000000
25%,-7.731126e-01,102.000000,-5.660315e-01,999.000000,0.000000
50%,-1.966547e-01,179.000000,-2.029780e-01,999.000000,0.000000
75%,6.680322e-01,319.000000,1.600755e-01,999.000000,0.000000
max,5.567924e+00,4918.000000,1.940191e+01,999.000000,6.000000


Let's take a look at the transformed `age` and `campaign` features and their statistics.

In [9]:
print("\nOriginal X_train 'age' and 'campaign' head:")
print(x_train[num_cols_demo].head())
print("\nScaled X_train 'age' and 'campaign' head:")
print(pd.DataFrame(x_train[num_cols_demo], columns=num_cols_demo, index=x_train.index).head())

print("\nMean of scaled 'age' (train):", x_train[num_cols_demo].iloc[:, 0].mean())
print("Std Dev of scaled 'campaign' (train):", x_train[num_cols_demo].iloc[:, 1].std())


Original X_train 'age' and 'campaign' head:
            age  campaign
16160 -1.157418  1.975343
2243  -0.100578  0.523129
30222 -0.484884 -0.566032
33523  1.820948 -0.202978
39916 -0.484884 -0.566032

Scaled X_train 'age' and 'campaign' head:
            age  campaign
16160 -1.157418  1.975343
2243  -0.100578  0.523129
30222 -0.484884 -0.566032
33523  1.820948 -0.202978
39916 -0.484884 -0.566032

Mean of scaled 'age' (train): -2.8193988752022287e-16
Std Dev of scaled 'campaign' (train): 1.0000173428950816


**2. Ordinal Feature: `education` (Ordinal Encoding with Imputation)**

- **Imputation**

In [10]:
ord_col_demo = ['education']

imputer_ord = SimpleImputer(strategy='most_frequent')

## Write your code here
x_train[ord_col_demo] = imputer_ord.fit_transform(x_train[ord_col_demo])
x_test[ord_col_demo] = imputer_ord.transform(x_test[ord_col_demo])
x_train['education']


,education
16160,high.school
2243,basic.9y
30222,university.degree
33523,basic.6y
39916,professional.course
...,...
38988,university.degree
1073,basic.4y
17127,professional.course
30087,high.school


- **Ordinal Encoding**

In [11]:
education_categories = [
    'illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school',
    'professional.course', 'university.degree', 'masters', 'doctorate'
]

In [12]:
ordinal_encoder = OrdinalEncoder(categories=[education_categories])

## Write your code here
x_train[ord_col_demo] = ordinal_encoder.fit_transform(x_train[ord_col_demo])
x_test[ord_col_demo] = ordinal_encoder.transform(x_test[ord_col_demo])

x_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
3819,-0.580960,blue-collar,married,3.0,no,no,no,telephone,may,fri,344,0.523129,999,0,nonexistent
18793,-0.677036,technician,married,6.0,no,yes,no,cellular,jul,thu,52,1.612289,999,0,nonexistent
2126,-0.580960,blue-collar,single,2.0,NaN,yes,no,telephone,may,mon,891,-0.566032,999,0,nonexistent
39389,1.532719,admin.,single,6.0,no,no,no,cellular,mar,wed,193,-0.566032,5,1,success
39000,-0.196655,technician,single,6.0,no,yes,no,cellular,dec,mon,329,-0.202978,999,0,nonexistent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7122,0.091574,housemaid,married,6.0,no,no,no,telephone,may,thu,665,-0.566032,999,0,nonexistent
1356,1.436643,housemaid,married,1.0,no,yes,no,telephone,may,thu,14,-0.566032,999,0,nonexistent
38476,-0.677036,admin.,single,4.0,no,no,no,cellular,oct,mon,251,-0.566032,999,0,nonexistent
19651,-0.388807,admin.,married,4.0,NaN,yes,no,cellular,aug,thu,165,0.160075,999,0,nonexistent


Let's take a look at the imputed and ordinal-encoded `education`.

In [13]:
print("\nOriginal X_train 'education' head:")
print(x_train[ord_col_demo].iloc[20:25])
print("\nImputed X_train 'education' head (after imputer.transform):")
print(pd.DataFrame(x_train[ord_col_demo], columns=ord_col_demo, index=x_train.index).iloc[20:25])
print("\nOrdinal Encoded X_train 'education' head:")
print(pd.DataFrame(x_train[ord_col_demo], columns=ord_col_demo, index=x_train.index).iloc[20:25])


Original X_train 'education' head:
       education
18089        5.0
26944        2.0
36957        6.0
12558        1.0
41160        6.0

Imputed X_train 'education' head (after imputer.transform):
       education
18089        5.0
26944        2.0
36957        6.0
12558        1.0
41160        6.0

Ordinal Encoded X_train 'education' head:
       education
18089        5.0
26944        2.0
36957        6.0
12558        1.0
41160        6.0


**3. Nominal Feature: `job` (One-Hot Encoding with Imputation)**

- **Imputation**

In [14]:
nom_col_demo = ['job']

imputer_nom = SimpleImputer(strategy='most_frequent')
imputer_nom.fit(x_train[nom_col_demo])

X_train_imputed_nom_demo = imputer_nom.transform(x_train[nom_col_demo])
X_test_imputed_nom_demo = imputer_nom.transform(x_test[nom_col_demo])

- **Nominal Encoding**

In [15]:
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

x_train_onehot = onehot_encoder.fit_transform(x_train[['job']])
x_test_onehot = onehot_encoder.transform(x_test[['job']])

x_train_onehot.shape

(28831, 12)

In [16]:
x_train_onehot

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [17]:
x_train = pd.concat([x_train.reset_index(drop=True),
                     pd.DataFrame(x_train_onehot, columns=onehot_encoder.get_feature_names_out(['job']))], axis=1)
x_train = x_train.drop(['job'], axis=1)

x_train

,age,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_nan
0,-1.157418,single,4.0,no,no,no,cellular,jul,tue,431,1.975343,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-0.100578,married,3.0,no,no,no,telephone,may,mon,249,0.523129,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.484884,married,6.0,no,yes,no,cellular,apr,thu,878,-0.566032,999,0,nonexistent,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.820948,divorced,2.0,NaN,yes,no,cellular,may,tue,216,-0.202978,999,0,nonexistent,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.484884,divorced,5.0,no,yes,yes,cellular,jun,wed,181,-0.566032,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28826,0.860185,single,6.0,no,no,yes,cellular,dec,mon,334,0.160075,999,0,nonexistent,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28827,1.052337,married,1.0,no,no,no,telephone,may,wed,353,0.523129,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28828,-0.004502,married,5.0,no,no,no,cellular,jul,fri,234,-0.566032,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
28829,-0.677036,single,4.0,no,yes,no,cellular,apr,mon,44,0.160075,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [18]:
x_test = pd.concat([x_test.reset_index(drop=True),
                    pd.DataFrame(x_test_onehot, columns=onehot_encoder.get_feature_names_out(['job']))], axis=1)

x_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_nan
0,-0.580960,blue-collar,married,3.0,no,no,no,telephone,may,fri,344,0.523129,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.677036,technician,married,6.0,no,yes,no,cellular,jul,thu,52,1.612289,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-0.580960,blue-collar,single,2.0,NaN,yes,no,telephone,may,mon,891,-0.566032,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.532719,admin.,single,6.0,no,no,no,cellular,mar,wed,193,-0.566032,5,1,success,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.196655,technician,single,6.0,no,yes,no,cellular,dec,mon,329,-0.202978,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12352,0.091574,housemaid,married,6.0,no,no,no,telephone,may,thu,665,-0.566032,999,0,nonexistent,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12353,1.436643,housemaid,married,1.0,no,yes,no,telephone,may,thu,14,-0.566032,999,0,nonexistent,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12354,-0.677036,admin.,single,4.0,no,no,no,cellular,oct,mon,251,-0.566032,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12355,-0.388807,admin.,married,4.0,NaN,yes,no,cellular,aug,thu,165,0.160075,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
print("\nOriginal X_train 'job' head:")
# The 'job' column was dropped from x_train in a previous step, so trying to access it directly will cause a KeyError.
# If you want to see the original imputed job values, use X_train_imputed_nom_demo.
# print(x_train[nom_col_demo].iloc[40:45]) # This line causes the KeyError
print("\nImputed X_train 'job' head (after imputer.transform):")
print(pd.DataFrame(X_train_imputed_nom_demo, columns=nom_col_demo, index=x_train.index).iloc[40:45])
print("\nOne-Hot Encoded X_train 'job' shape:", x_train_onehot.shape)
print("First 5 rows of One-Hot Encoded X_train 'job':")
print(pd.DataFrame(x_train_onehot, columns=onehot_encoder.get_feature_names_out(nom_col_demo), index=x_train.index).iloc[40:45])


Original X_train 'job' head:

Imputed X_train 'job' head (after imputer.transform):
            job
40     services
41  blue-collar
42       admin.
43       admin.
44  blue-collar

One-Hot Encoded X_train 'job' shape: (28831, 12)
First 5 rows of One-Hot Encoded X_train 'job':
    job_admin.  job_blue-collar  job_entrepreneur  job_housemaid  job_management  job_retired  job_self-employed  job_services  job_student  job_technician  job_unemployed  job_nan
40         0.0              0.0               0.0            0.0             0.0          0.0                0.0           1.0          0.0             0.0             0.0      0.0
41         0.0              1.0               0.0            0.0             0.0          0.0                0.0           0.0          0.0             0.0             0.0      0.0
42         1.0              0.0               0.0            0.0             0.0          0.0                0.0           0.0          0.0             0.0             0.0      0.

### **Exercise 1: Apply All Preprocessing & Train Logistic Regression**

Now, it's your turn to apply these preprocessing steps to *all* relevant columns and then train a Logistic Regression model.

**Instructions:**

1.  Look at the Variable Table in [this link](https://archive.ics.uci.edu/dataset/222/bank+marketing).
2. Make lists for `numerical_features`, `ordinal_features`, and `nominal_features`.
3. Preprocess the features. It is safer to make a copy of `X_train` using:
   ```
   X_train_copy = X_train.copy()
   X_test_copy = X_test.copy()
   ```
   and preprocess `X_train_copy` instead.

   **For nominal features, concat the one-hot encoded features using [`pd.concat(..., axis=1)`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) and drop the old nominal features from the dataframe.**
4. Train Logistic Regression on the preprocessed `X_train_copy` and `y_train`.
5. Evaluate the Model:
    *   Make predictions on the preprocessed `X_test_copy`.
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?


In [20]:
bank_url = 'https://raw.githubusercontent.com/donlap/ds352-labs/main/bank.csv'

df = pd.read_csv(bank_url, sep=';', na_values=['unknown'])
df = df.drop(["emp.var.rate", "cons.price.idx", "cons.conf.idx",	"euribor3m", "nr.employed"], axis=1)
print("Shape of the dataset:", df.shape)
df.head()


Shape of the dataset: (41188, 16)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,no
1,57,services,married,high.school,NaN,no,no,telephone,may,mon,149,1,999,0,nonexistent,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,no


In [21]:
numerical_features = ['age', 'duration', 'campaign', 'pdays', 'previous']
ordinal_features = ['education']
nominal_features = ['marital', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'day_of_week'] # Updated to include all nominal features

In [22]:
x_train_copy = x_train.copy()
x_test_copy = x_test.copy()

**numericalm**

In [23]:
scaler = StandardScaler()

x_train_copy[numerical_features] = scaler.fit_transform(x_train_copy[numerical_features]) # Apply to all numerical_features
x_test_copy[numerical_features] = scaler.transform(x_test_copy[numerical_features]) # Apply to all numerical_features

In [24]:
x_train_copy.describe()

,age,education,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_nan
count,28831.000000,28831.000000,2.883100e+04,2.883100e+04,2.883100e+04,2.883100e+04,28831.000000,28831.000000,28831.000000,28831.000000,28831.000000,28831.000000,28831.000000,28831.000000,28831.000000,28831.000000,28831.000000,28831.000000
mean,0.000000,4.241372,2.070188e-17,7.886430e-18,1.725157e-16,-2.957411e-17,0.251292,0.225972,0.035032,0.024904,0.072457,0.041344,0.034373,0.096597,0.021643,0.164406,0.024279,0.007700
std,1.000017,1.644247,1.000017e+00,1.000017e+00,1.000017e+00,1.000017e+00,0.433764,0.418228,0.183863,0.155835,0.259247,0.199089,0.182188,0.295414,0.145519,0.370650,0.153918,0.087413
min,-2.214257,0.000000,-9.913806e-01,-5.660315e-01,-5.137890e+00,-3.493294e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-0.773113,3.000000,-6.001335e-01,-5.660315e-01,1.958461e-01,-3.493294e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-0.196655,4.000000,-3.047803e-01,-2.029780e-01,1.958461e-01,-3.493294e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.668032,6.000000,2.322255e-01,1.600755e-01,1.958461e-01,-3.493294e-01,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.567924,6.000000,1.787287e+01,1.940191e+01,1.958461e-01,1.180211e+01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


**Ordinal Encoding**

In [25]:
x_train_copy['education'].unique()

array([4., 3., 6., 2., 5., 1., 0.])

In [26]:
x_train_copy['education'] = x_train_copy['education'].astype(float)
x_test_copy['education'] = x_test_copy['education'].astype(float)

In [27]:
x_train_copy

,age,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_nan
0,-1.157418,single,4.0,no,no,no,cellular,jul,tue,0.661830,1.975343,0.195846,-0.349329,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-0.100578,married,3.0,no,no,no,telephone,may,mon,-0.036277,0.523129,0.195846,-0.349329,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.484884,married,6.0,no,yes,no,cellular,apr,thu,2.376413,-0.566032,0.195846,-0.349329,nonexistent,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.820948,divorced,2.0,NaN,yes,no,cellular,may,tue,-0.162857,-0.202978,0.195846,-0.349329,nonexistent,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.484884,divorced,5.0,no,yes,yes,cellular,jun,wed,-0.297109,-0.566032,0.195846,-0.349329,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28826,0.860185,single,6.0,no,no,yes,cellular,dec,mon,0.289762,0.160075,0.195846,-0.349329,nonexistent,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28827,1.052337,married,1.0,no,no,no,telephone,may,wed,0.362641,0.523129,0.195846,-0.349329,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28828,-0.004502,married,5.0,no,no,no,cellular,jul,fri,-0.093814,-0.566032,0.195846,-0.349329,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
28829,-0.677036,single,4.0,no,yes,no,cellular,apr,mon,-0.822607,0.160075,0.195846,-0.349329,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [28]:
x_train_copy.columns.tolist()

['age',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'job_admin.',
 'job_blue-collar',
 'job_entrepreneur',
 'job_housemaid',
 'job_management',
 'job_retired',
 'job_self-employed',
 'job_services',
 'job_student',
 'job_technician',
 'job_unemployed',
 'job_nan']

In [29]:
x_test_copy.columns.tolist()

['age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'job_admin.',
 'job_blue-collar',
 'job_entrepreneur',
 'job_housemaid',
 'job_management',
 'job_retired',
 'job_self-employed',
 'job_services',
 'job_student',
 'job_technician',
 'job_unemployed',
 'job_nan']

**Nominal Feature**

In [30]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split


x_train_copy, x_test_copy, y_train_exercise, y_test_exercise = train_test_split(x.copy(), y.copy(), test_size=0.3, random_state=42)

# --- Drop 'duration' from here, as the original 'x' still contains it ---
x_train_copy = x_train_copy.drop('duration', axis=1)
x_test_copy = x_test_copy.drop('duration', axis=1)

# Define feature lists for this exercise (updated with 'duration' removed from numerical)
numerical_features = ['age', 'campaign', 'pdays', 'previous']
ordinal_features = ['education']
nominal_features = ['job', 'marital', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']

# --- 1. Numerical Feature Preprocessing (Standard Scaling) ---
scaler_exercise = StandardScaler()
x_train_copy[numerical_features] = scaler_exercise.fit_transform(x_train_copy[numerical_features])
x_test_copy[numerical_features] = scaler_exercise.transform(x_test_copy[numerical_features])

# --- 2. Ordinal Feature Preprocessing (Imputation & Encoding) ---

# Impute missing values for ordinal features
imputer_ord_exercise = SimpleImputer(strategy='most_frequent')
x_train_copy[ordinal_features] = imputer_ord_exercise.fit_transform(x_train_copy[ordinal_features])
x_test_copy[ordinal_features] = imputer_ord_exercise.transform(x_test_copy[ordinal_features])

# Define education categories (re-defining for robustness)
education_categories_correct = [
    'illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school',
    'professional.course', 'university.degree', 'masters', 'doctorate'
]

ord_enc_exercise = OrdinalEncoder(categories=[education_categories_correct], handle_unknown='use_encoded_value', unknown_value=-1)
x_train_copy[ordinal_features] = ord_enc_exercise.fit_transform(x_train_copy[ordinal_features])
x_test_copy[ordinal_features] = ord_enc_exercise.transform(x_test_copy[ordinal_features])

# --- 3. Nominal Feature Preprocessing (Imputation & One-Hot Encoding) ---

# Imputation for nominal features
imputer_nom_exercise = SimpleImputer(strategy='most_frequent')
x_train_copy[nominal_features] = imputer_nom_exercise.fit_transform(x_train_copy[nominal_features])
x_test_copy[nominal_features] = imputer_nom_exercise.transform(x_test_copy[nominal_features])

# One-Hot Encoding for nominal features
ohe_exercise = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')

x_train_ohe = pd.DataFrame(ohe_exercise.fit_transform(x_train_copy[nominal_features]), index=x_train_copy.index)
x_test_ohe = pd.DataFrame(ohe_exercise.transform(x_test_copy[nominal_features]), index=x_test_copy.index)

x_train_ohe.columns = ohe_exercise.get_feature_names_out(nominal_features)
x_test_ohe.columns = ohe_exercise.get_feature_names_out(nominal_features)

# Concat one-hot encoded features and drop original nominal columns
x_train_copy = pd.concat([x_train_copy.drop(columns=nominal_features), x_train_ohe], axis=1)
x_test_copy = pd.concat([x_test_copy.drop(columns=nominal_features), x_test_ohe], axis=1)

In [31]:
x_train_copy

,age,education,campaign,pdays,previous,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_married,marital_single,default_yes,housing_yes,loan_yes,contact_telephone,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_nonexistent,poutcome_success
39075,-1.053709,6.0,0.154139,0.193350,1.673800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
34855,-1.053709,6.0,0.517478,0.193350,-0.349144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7107,0.477399,2.0,-0.209199,0.193350,-0.349144,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
31614,-0.575238,6.0,-0.572537,0.193350,1.673800,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
34878,-0.766626,4.0,2.334170,0.193350,-0.349144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,1.721425,5.0,-0.209199,0.193350,-0.349144,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
11284,-0.288155,6.0,-0.572537,0.193350,-0.349144,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
38158,-0.479543,4.0,-0.572537,-5.182867,1.673800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
860,-0.001072,6.0,-0.209199,0.193350,-0.349144,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [32]:
x_test_copy

,age,education,campaign,pdays,previous,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_married,marital_single,default_yes,housing_yes,loan_yes,contact_telephone,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_nonexistent,poutcome_success
32884,1.625730,4.0,-0.572537,0.193350,1.673800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3169,1.434342,6.0,-0.209199,0.193350,-0.349144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
32206,-0.670932,3.0,-0.572537,0.193350,1.673800,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9403,-0.383849,4.0,0.517478,0.193350,-0.349144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
14020,-1.245098,4.0,-0.209199,0.193350,-0.349144,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15908,0.573094,4.0,-0.209199,0.193350,-0.349144,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
28222,-0.192461,4.0,-0.572537,-5.193674,1.673800,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
14194,-1.340792,4.0,-0.209199,0.193350,-0.349144,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
19764,1.051565,5.0,0.154139,0.193350,-0.349144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


**Train Logistic Regression and Evaluate the Model**

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# 1) Train model
lr = LogisticRegression(max_iter=1000, solver='lbfgs')
lr.fit(x_train_copy, y_train)

# 2) Predict on test set
y_pred = lr.predict(x_test_copy)

# 3) Evaluation
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      1.00      0.94     10934
           1       0.00      0.00      0.00      1423

    accuracy                           0.88     12357
   macro avg       0.44      0.50      0.47     12357
weighted avg       0.78      0.88      0.83     12357



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Part 2: Fashion-MNIST Dataset - Image Classification

### Load Fashion-MNIST Dataset

The Fashion-MNIST dataset consists of 28x28 grayscale images of fashion items.

In [34]:
(fm_X_train, fm_y_train), (fm_X_test, fm_y_test) = fashion_mnist.load_data()

print(f"Fashion-MNIST Train data shape: {fm_X_train.shape}")
print(f"Fashion-MNIST Train labels shape: {fm_y_train.shape}")
print(f"Fashion-MNIST Test data shape: {fm_X_test.shape}")
print(f"Fashion-MNIST Test labels shape: {fm_y_test.shape}")

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Fashion-MNIST Train data shape: (60000, 28, 28)
Fashion-MNIST Train labels shape: (60000,)
Fashion-MNIST Test data shape: (10000, 28, 28)
Fashion-MNIST Test labels shape: (10000,)


In [35]:
print(f"First image {fm_X_train[0]}")
print(f"First label {fm_y_train[0]}")

First image [[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   1   0   0  13  73   0
    0   1   4   0   0   0   0   1   1   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3   0  36 136 127  62
   54   0   0   0   1   3   4   0   0   3]
 [  0   0   0   0   0   0   0   0   0   0   0   0   6   0 102 204 176 134
  144 123  23   0   0   0   0  12  10   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0 155 236 207 178
  107 156 161 109  64  23  77 130  72  15]
 [  0   0   0   0   0   0   0   0   0   0   0   1   0  69 207 223 218 216
  216 163 127 121 122 146 141  88 172  66]
 [  0   0   0   0   0   0   0   0   0   1   1   1   

### Visualize Fashion-MNIST Images

Let's see what these images look like.

In [36]:
fashion_mnist_class_names = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]

# Visualize the images
## Write your code here



### **Exercise 2: Preprocessing Images (Flatten and Scale)**

Images are 2D arrays (matrices of pixels) and pixel values are integers from 0-255. For Logistic Regression, we need:
*  **Flattening:** Convert each 28x28 image into a 1D array of 784 features.
*  **Scaling:** Normalize pixel values from [0, 255] to [0, 1].

**Instructions:**

1.   **Flatten:** Use the `.reshape()` method (see [documentation](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.reshape.html)). For `fm_X_train_binary` (shape `(num_samples, 28, 28)`), you want to reshape it to `(num_samples, 28*28)`.
2.  **Scale:** Divide the flattened pixel values by 255.0 to get values between 0 and 1.
3.   **Train Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
    *   Fit the model on your *processed* `fm_X_train_scaled` and `fm_y_train`.
4.   **Make Predictions:** Use `predict()` to make predictions on the *processed* `fm_X_test_scaled`.
5.   **Print Classification Report:** Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?
6.   **Visualize Misclassifications:**
    *   Find the indices in `fm_X_test_binary` where your model made incorrect predictions (i.e., `fm_y_pred != fm_y_test`).
    *   Select 5 of these misclassified images.
    *   Plot these images (using `plt.imshow`). For each image, print its true label and its predicted label.

In [37]:
# --- YOUR CODE FOR EXERCISE 2 STARTS HERE ---





## Part 3: 20 Newsgroups Dataset - Text Classification

### Load 20 Newsgroups Dataset

The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics.

In [38]:
news_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
news_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)

X_train_news, y_train_news = news_train.data, news_train.target
X_test_news, y_test_news = news_test.data, news_test.target

print(f"Number of training documents: {len(X_train_news)}")
print(f"Number of test documents: {len(X_test_news)}")
print(f"Categories: {news_train.target_names}")

Number of training documents: 11314
Number of test documents: 7532
Categories: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


### Explore Sample Document

In [39]:
# Print the first document and its class
## Write your code here



### Preprocessing: Text Vectorization Demonstration with `TfidfVectorizer`

$$
\text{TF-IDF}(t, d, D) = \text{TF}(t, d) \times \text{IDF}(t, D)
$$

Where:

$$
\text{TF}(t, d) = \frac{\text{number of word }t\text{ in } d}{\text{number of words in } d} \quad \text{ and } \quad
\text{IDF}(t, D) = \log\left(\frac{\text{total number of documents}}{\text{number of documents that contain word }t}\right).
$$

In [40]:
sample_sentences = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?"
]

vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the sample sentences
sample_vec_output_sparse = # Write your code here

sample_vec_output_dense = sample_vec_output_sparse.toarray()

print(vectorizer.vocabulary_)
print(vectorizer.get_feature_names_out())
print(sample_vec_output_dense)

SyntaxError: invalid syntax (ipython-input-2984329282.py, line 11)

### **Exercise 3: Apply TF-IDF Vectorization to Full Dataset**

Now, apply `TfidfVectorizer` to the actual training and testing datasets for the 20 Newsgroups classification task.

**Instructions:**

1.  **Initialize `TfidfVectorizer`:**
    *   Initialize `TfidfVectorizer`. Use `stop_words='english'` to remove common words.
2.  **Fit and Transform Training Data:**
    *   Call `fit_transform()` on `X_train_news` to learn the vocabulary and transform the training text into TF-IDF features. Store the result in `X_train_vec`.
3.  **Transform Test Data:**
    *   Call `transform()` on `X_test_news` using the *already fitted* vectorizer. Store the result in `X_test_vec`. **Crucially, do not call `fit_transform()` on the test data!** This would cause data leakage.
4.  **Initialize Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
5.  **Train the Model:**
    *   Fit the model on your `X_train_vec` and `y_train_news`.
6.  **Make Predictions:**
    *   Make predictions using `predict()` on the `X_test_vec`.
7.  **Evaluate the Model:**
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?

In [ ]:
# --- YOUR CODE FOR EXERCISE 3 STARTS HERE ---


